##Pandahouse

###Установка ClickHouse

In [1]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20210119~18.04.1).
dirmngr is already the newest version (2.2.4-1ubuntu1.4).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 1,692 B of archives.
After this operation, 154 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.13 [1,692 B]
Fetched 1,692 B in 0s (9,729 B/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

###Загрузка датасетов

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_ratings = pd.read_csv('drive/MyDrive/dataset/movies_rating.csv')

In [4]:
df_ratings['date'] = pd.to_datetime(df_ratings['timestamp'],unit='s')

In [5]:
df_ratings['date'] = df_ratings['date'].dt.date

In [6]:
df_ratings = df_ratings[['date','userId','movieId','rating']]
df_ratings.columns = ['date','user_id','movie_id','rating']

In [7]:
df_ratings.head()

,date,user_id,movie_id,rating
0,2009-12-14,1,31,2.5
1,2009-12-14,1,1029,3.0
2,2009-12-14,1,1061,3.0
3,2009-12-14,1,1129,2.0
4,2009-12-14,1,1172,4.0


In [8]:
df_ratings.isnull().sum()

date        0
user_id     0
movie_id    0
rating      0
dtype: int64

In [9]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   date      100004 non-null  object 
 1   user_id   100004 non-null  int64  
 2   movie_id  100004 non-null  int64  
 3   rating    100004 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.1+ MB


In [10]:
usecols = ['id','title']
df_movies = pd.read_csv('drive/MyDrive/dataset/movies.csv',usecols=usecols)
df_movies.columns = ['movie_id', 'title']

In [11]:
df_movies.head()

,movie_id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [12]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  45466 non-null  object
 1   title     45460 non-null  object
dtypes: object(2)
memory usage: 710.5+ KB


In [13]:
df_movies.movie_id = pd.to_numeric(df_movies.movie_id , errors='coerce').convert_dtypes()

In [14]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  45463 non-null  Int64 
 1   title     45460 non-null  object
dtypes: Int64(1), object(1)
memory usage: 754.9+ KB


In [15]:
df_movies.isnull().sum()

movie_id    3
title       6
dtype: int64

In [16]:
df_movies = df_movies.dropna()

In [17]:
df_movies.isnull().sum()

movie_id    0
title       0
dtype: int64

In [18]:
df_movies['movie_id'] = df_movies['movie_id'].astype('int64')

In [19]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45460 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  45460 non-null  int64 
 1   title     45460 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


###Добавление новой БД и таблиц

In [20]:
!pip install clickhouse-driver

     |████████████████████████████████| 481kB 13.3MB/s 


In [21]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('default',), ('system',)]

In [22]:
client.execute('CREATE DATABASE db_movies')

[]

In [23]:
client.execute('SHOW DATABASES')

[('db_movies',), ('default',), ('system',)]

In [24]:
client = Client(host='localhost', user='default', port='9000', database='db_movies')

In [25]:
client.execute('DROP TABLE IF EXISTS ratings')
client.execute('CREATE TABLE ratings (date Date, user_id Int64, movie_id Int64, rating Float32  ) ENGINE = Memory')

[]

In [26]:
client.execute('DROP TABLE IF EXISTS movies')
client.execute('CREATE TABLE movies (movie_id Int64, title String ) ENGINE = Memory')

[]

In [27]:
client.execute('SHOW TABLES FROM db_movies')

[('movies',), ('ratings',)]

### Загрузка и чтение записей из БД

In [28]:
!pip install pandahouse

  Created wheel for pandahouse: filename=pandahouse-0.2.7-py2.py3-none-any.whl size=5922 sha256=1233e1516259ef16e94d39a58b86c8eeeb36b135621b33c8be687416216d1887
  Stored in directory: /root/.cache/pip/wheels/34/cd/4e/f42f963ab3b7a59b470cc45b2d3dcee0c310731feb780ca2ff
Successfully built pandahouse


In [29]:
from pandahouse import to_clickhouse, read_clickhouse

In [30]:
connection = {'host':'http://localhost:8123', 'database': 'db_movies'}

### Обращать внимание на проблемы конвертации типов данных (https://github.com/kszucs/pandahouse/blob/master/pandahouse/convert.py)

In [31]:
df_ratings = to_clickhouse(df_ratings, table='ratings', index=False, connection=connection)

In [32]:
df_sql_ratings = read_clickhouse('SELECT * FROM db_movies.ratings', connection=connection)

In [33]:
df_sql_ratings.head()

,date,user_id,movie_id,rating
0,2009-12-14,1,31,2.5
1,2009-12-14,1,1029,3.0
2,2009-12-14,1,1061,3.0
3,2009-12-14,1,1129,2.0
4,2009-12-14,1,1172,4.0


In [34]:
df_movies = to_clickhouse(df_movies, table='movies', index=False, connection=connection)

In [35]:
df_sql_movies = read_clickhouse('SELECT * FROM db_movies.movies', connection=connection)

In [36]:
df_sql_movies.head()

,movie_id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


###Запросы к БД

In [37]:
read_clickhouse('SELECT r.*,  \
                      m.title \
                FROM  \
                    (SELECT r.movie_id,  \
                            count(r.rating) as count,  \
                            avg(r.rating) as avg \
                    FROM db_movies.ratings as r\
                    GROUP BY r.movie_id  \
                    ORDER BY count(r.rating) DESC, \
                            avg(r.rating) DESC) as r  \
                    LEFT JOIN db_movies.movies as m ON r.movie_id = m.movie_id \
                LIMIT 10', connection=connection)

,movie_id,count,avg,title
0,356,341,4.054252,
1,296,324,4.256173,Terminator 3: Rise of the Machines
2,318,311,4.487138,The Million Dollar Hotel
3,593,304,4.138158,Solaris
4,260,291,4.221649,The 39 Steps
5,480,274,3.706204,Monsoon Wedding
6,2571,259,4.183398,
7,1,247,3.872470,
8,527,244,4.303279,Once Were Warriors
9,589,237,4.006329,
